### Script to convert geolocation into correct format

In [1]:
import geopandas as gpd
import pandas as pd
import os

In [2]:
%config Completer.use_jedi = False

### Settings

In [3]:
INFILE = 'nav.txt'
NEW_CRS = 'EPSG:32608'
H_ACC = 1
V_ACC = 1

#### Load images 

In [7]:
df = pd.read_csv(INFILE, sep='\t')

In [10]:
df.head(2)

,File,Date,Time,msOfDay,Lat[deg],Lon[deg],Alt[m],Easting[m],Northing[m],Zone,Roll[deg],Pitch[deg],Yaw[deg],Omega[deg],Phi[deg],Kappa[deg]
0,N:/response/Restricted_Airborne/MACs/Canada/1_...,2018-08-22,20:53:13.074,75193074,68.731418,-133.588401,910.671063,557139.74,7625070.46,8W,0.0385,3.1979,225.6143,-8.481340,-3.562697,132.369845
1,N:/response/Restricted_Airborne/MACs/Canada/1_...,2018-08-22,21:12:21.074,76341074,68.705597,-133.697328,905.961692,552792.10,7622094.69,8W,1.0405,2.6993,228.0759,-9.043587,-4.231675,129.822436


#### Change image suffixes 

In [14]:
images = df['File '].str.replace('.macs', '.tif')

In [36]:
images = images.apply(lambda x: x.strip().split('/')[-1])

#### Load coordinates to geom an reproject to selected crs 

In [37]:
#gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['latitude [decimal degrees]'], df['longitude [decimal degrees]']))

In [38]:
series = gpd.GeoSeries(gpd.points_from_xy(df['Lon[deg] '], df['Lat[deg] ']), crs='EPSG:4326')
series_new = series.to_crs(crs=NEW_CRS)

#### Fill Table 

In [39]:
df['imagename_tif'] = images
df['x'] = series_new.geometry.x
df['y'] = series_new.geometry.y
df['horizontal_accuracy'] = H_ACC
df['vertical_accuracy'] = V_ACC

#### Create final structure

In [40]:
df.columns

Index(['File ', 'Date ', 'Time ', 'msOfDay ', 'Lat[deg] ', 'Lon[deg] ',
       'Alt[m] ', 'Easting[m] ', 'Northing[m] ', 'Zone ', 'Roll[deg] ',
       'Pitch[deg] ', 'Yaw[deg] ', 'Omega[deg] ', 'Phi[deg] ', 'Kappa[deg]',
       'imagename_tif', 'x', 'y', 'horizontal_accuracy', 'vertical_accuracy'],
      dtype='object')

In [41]:
df_new = df[['imagename_tif', 'x', 'y', 'Alt[m] ', 'Omega[deg] ',
       'Phi[deg] ', 'Kappa[deg]', 'horizontal_accuracy', 'vertical_accuracy']]

In [42]:
df_new

,imagename_tif,x,y,Alt[m],Omega[deg],Phi[deg],Kappa[deg],horizontal_accuracy,vertical_accuracy
0,02268_004759891_1500.tif,557139.742807,7.625070e+06,910.671063,-8.481340,-3.562697,132.369845,1,1
1,04564_010707893_1500.tif,552792.099006,7.622095e+06,905.961692,-9.043587,-4.231675,129.822436,1,1
2,02267_004759391_1500.tif,557164.148860,7.625091e+06,910.485519,-8.593521,-3.610940,132.184241,1,1
3,02269_004800391_1500.tif,557115.402391,7.625050e+06,910.896608,-8.363080,-3.505972,132.406828,1,1
4,02266_004758891_1500.tif,557188.604512,7.625111e+06,910.406976,-8.541458,-3.776054,131.920429,1,1
...,...,...,...,...,...,...,...,...,...
46212,18957_030706517_1800.tif,549366.853136,7.623337e+06,892.885933,-0.742938,2.181139,130.806678,1,1
46213,18963_030709517_1800.tif,549213.952959,7.623220e+06,892.711348,0.044048,2.691989,131.272922,1,1
46214,18965_030710517_1800.tif,549163.572213,7.623180e+06,892.465831,-0.094110,2.670298,131.764798,1,1
46215,18959_030707517_1800.tif,549315.595904,7.623298e+06,892.916433,-0.238094,2.582737,130.788446,1,1


#### Export file 

In [43]:
df_new.to_csv('out.txt', sep='\t', header=False, index=False)

In [44]:
with open('out.txt', 'r') as f:
    l = f.readlines()

In [45]:
newlines = [NEW_CRS + '\n']  +l

In [46]:
with open('geo.txt', 'w') as f_new:
    f_new.writelines(newlines)

In [47]:
os.remove('out.txt')